In [1]:
!pip install tf-keras

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

import random
import os
import ast
import json
import spacy
from spacy.training import Example

from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor

import pickle
from tqdm import tqdm
import gc

from torch.utils.data import Dataset, DataLoader
import time


C:\drivers\python_inter\Lib\site-packages\cupy\_environment.py:215: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [3]:
MAX_LEN = 128
BATCH_SIZE = 32
EMB_DIM = 128
LATENT_DIM = 64
N_HEADS = 8
FF_DIM = 512
NUM_LAYERS = 2
VOCAB_SPECIAL = ['<pad>', '<bos>', '<eos>', '<unk>']

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используем устройство: {DEVICE}")
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))

Используем устройство: cuda
True
12.8
NVIDIA GeForce RTX 3060 Laptop GPU


In [4]:
# Загружаем датасет
df = pd.read_csv("data/polyOne_aa.csv")

# Проверяем наличие столбца SMILES (в вашем датасете он называется именно "smiles")
if "smiles" not in df.columns:
    raise ValueError("Столбец 'smiles' не найден в датасете. Проверьте название столбца.")

# Удаляем строки с пустыми SMILES
df = df.dropna(subset=["smiles"])

# Полный маппинг всех свойств из датасета с их расшифровкой
with open("data/user_request_dataset/property_mapping.json", "r", encoding="utf-8") as f:
    PROPERTY_MAPPING = json.load(f)

# Все числовые признаки из датасета (в оригинальных названиях)
NUM_FEATURES = list(PROPERTY_MAPPING.keys())
print(NUM_FEATURES)

['PROPERTY_MAPPING']


In [5]:
def categorize_values(series, property_name):
    # Проверка на пропущенные значения
    nan_count = series.isna().sum()
    if nan_count > 0:
        print(f"⚠️ ВНИМАНИЕ: В колонке {property_name} найдено {nan_count} пропущенных значений (NaN). Они будут исключены из анализа.")
        series = series.dropna()
    
    # Проверка на единственное уникальное значение
    if series.nunique() == 1:
        print(f"⚠️ ВНИМАНИЕ: В колонке {property_name} все значения одинаковы ({series.iloc[0]}). Все данные будут отнесены к категории 'среднее'.")
        categories = pd.Series(['среднее'] * len(series), index=series.index)
        return categories, ['низкое', 'среднее', 'высокое']
    
    # Используем квантили для балансировки категорий
    low_bound = series.quantile(0.33)
    high_bound = series.quantile(0.67)
    
    # Создание категорий
    bins = [-np.inf, low_bound, high_bound, np.inf]
    labels = ['низкое', 'среднее', 'высокое']
    categories = pd.cut(series, bins=bins, labels=labels)
    
    # Проверка баланса категорий
    value_counts = categories.value_counts().sort_index()
    total = len(categories)
    
    # Вывод информации
    print(f"Диапазон значений для '{property_name}': {series.min():.4f} - {series.max():.4f}")
    print(f"Пороги категорий: низкое <= {low_bound:.4f}, среднее <= {high_bound:.4f}, высокое > {high_bound:.4f}")
    print(f"Распределение категорий: низкое={value_counts['низкое']} ({value_counts['низкое']/total:.1%}), "
          f"среднее={value_counts['среднее']} ({value_counts['среднее']/total:.1%}), "
          f"высокое={value_counts['высокое']} ({value_counts['высокое']/total:.1%})")
    
    return categories, labels

# Определяем колонки для обработки
common_columns = ['Egc','Egb','Eib','CED','Ei','Eea','nc','ne','epse_6.0','epsc','epse_3.0','epse_1.78','epse_15.0','epse_4.0','epse_5.0','epse_2.0','epse_9.0','epse_7.0','TSb','TSy','epsb','YM','permCH4','permCO2','permH2','permO2','permN2','permHe','Eat','rho','LOI','Xc','Xe','Cp','Td','Tg','Tm']

In [48]:
# Применяем категоризацию ко всем колонкам
for col in common_columns:
    if col in df:
        print(f"\nОбработка колонки '{col}' в df:")
        values = df[col]
        categories, labels = categorize_values(values, {col})
        df[f'{col}_category'] = categories
        
        # Проверка распределения категорий
        print("Распределение категорий:")
        print(df[f'{col}_category'].value_counts(dropna=False))
    else:
        print(f"⚠️ Колонка '{col}' отсутствует")


Обработка колонки 'Egc' в df:
Диапазон значений для '{'Egc'}': 0.4376 - 7.4522
Пороги категорий: низкое <= 3.1779, среднее <= 3.6253, высокое > 3.6253
Распределение категорий: низкое=165000 (33.0%), среднее=170000 (34.0%), высокое=165000 (33.0%)
Распределение категорий:
Egc_category
среднее    170000
низкое     165000
высокое    165000
Name: count, dtype: int64

Обработка колонки 'Egb' в df:
Диапазон значений для '{'Egb'}': 0.7974 - 7.6678
Пороги категорий: низкое <= 2.7763, среднее <= 3.2696, высокое > 3.2696
Распределение категорий: низкое=165000 (33.0%), среднее=170000 (34.0%), высокое=165000 (33.0%)
Распределение категорий:
Egb_category
среднее    170000
низкое     165000
высокое    165000
Name: count, dtype: int64

Обработка колонки 'Eib' в df:
Диапазон значений для '{'Eib'}': 2.0447 - 5.0319
Пороги категорий: низкое <= 3.2163, среднее <= 3.4721, высокое > 3.4721
Распределение категорий: низкое=165000 (33.0%), среднее=170000 (34.0%), высокое=165000 (33.0%)
Распределение категорий

In [7]:
# Глобальные конфигурационные данные (вынесены для удобства)
GENDER_SYNONYMS = {
    'm': {  # мужской род (коэффициент, радиус)
        'низкая': ['низкий', 'маленький', 'небольшой', 'скромный', 'невысокий'],
        'средняя': ['средний', 'умеренный', 'нормальный', 'стандартный'],
        'высокая': ['высокий', 'большой', 'значительный', 'превосходный', 'выдающийся']
    },
    'f': {  # женский род (активность, плотность)
        'низкая': ['низкая', 'маленькая', 'небольшая', 'скромная', 'невысокая'],
        'средняя': ['средняя', 'умеренная', 'нормальная', 'стандартная'],
        'высокая': ['высокая', 'большая', 'значительная', 'превосходная', 'выдающаяся']
    },
    'n': {  # средний род (свойство, состояние)
        'низкая': ['низкое', 'маленькое', 'небольшое', 'скромное', 'невысокое'],
        'средняя': ['среднее', 'умеренное', 'нормальное', 'стандартное'],
        'высокая': ['высокое', 'большое', 'значительное', 'превосходное', 'выдающееся']
    }
}

In [8]:
import json

with open("data/user_request_dataset/param_config.json", "r", encoding="utf-8") as f:
    PARAM_CONFIG = json.load(f)

In [9]:
# Шаблоны для начала запроса
with open("data/user_request_dataset/start_phrases.json", "r", encoding="utf-8") as f:
    data = json.load(f)
START_PHRASES = data["START_PHRASES"]

In [10]:
# Цели использования
with open("data/user_request_dataset/purposes.json", "r", encoding="utf-8") as f:
    data = json.load(f)
PURPOSES = data["PURPOSES"]

In [11]:
# Соединители для параметров
CONNECTORS = [
        ", а также ", ", при этом ", ", но при этом ", ", и при этом ", 
        ", дополнительно ", ", что важно ", ", что критично ", ", еще "
        ", что необходимо ", ", что требуется ", ", что важно для ", ", и ", ", который имеет "
    ]

In [13]:
def calculate_thresholds_sampled(df, sample_size=100_000):
    """
    Рассчитывает пороги на основе выборки данных для экономии памяти
    """
    thresholds = {}
    for param in df.columns:
        if param not in PARAM_CONFIG:
            continue
            
        # Берем выборку вместо полного датафрейма
        sample = df[param].dropna()
        if len(sample) > sample_size:
            sample = sample.sample(n=sample_size, random_state=42)
            
        if len(sample) < 10:  # Минимум 10 значений для расчета порогов
            continue
            
        thresholds[param] = {
            'low': sample.quantile(0.33),
            'high': sample.quantile(0.67)
        }
    
    return thresholds

In [14]:
def get_value_category_safe(value, thresholds, param):
    """Безопасное определение категории значения"""
    if pd.isna(value) or param not in thresholds:
        return None
        
    if value <= thresholds[param]['low']:
        return 'низкая'
    elif value >= thresholds[param]['high']:
        return 'высокая'
    else:
        return 'средняя'

In [15]:
def generate_batch_memory_efficient(batch_size, df, param_config, thresholds, min_params=2, max_params=5):
    """
    Генерирует батч примеров с минимальным использованием памяти
    
    Вместо предзагрузки всего датафрейма, используем выборки
    """
    results = []
    valid_params = [p for p in param_config.keys() if p in df.columns]
    
    if not valid_params:
        return []
    
    # Предзагружаем конфиги для всех параметров один раз
    param_templates = {}
    for param in valid_params:
        config = param_config[param]
        gender = config['gender']
        param_templates[param] = {
            'names': config['names'],
            'phrases': config['phrases'],
            'codes': config['codes'],
            'gender': gender,
            'adjectives': GENDER_SYNONYMS[gender]
        }
    
    for _ in range(batch_size):
        # Выбираем случайные параметры
        num_params = random.randint(min_params, min(max_params, len(valid_params)))
        selected_params = random.sample(valid_params, num_params)
        
        param_phrases = []
        annotations = []
        
        for param in selected_params:
            config = param_templates[param]
            gender = config['gender']
            
            # Получаем выборку данных для этого параметра
            param_data = df[param].dropna()
            if len(param_data) == 0:
                continue
                
            # Берем случайное значение из данных
            idx = random.choice(param_data.index)
            value = param_data.loc[idx]
            
            # Определяем категорию
            category = get_value_category_safe(value, thresholds, param)
            if not category:
                continue
                
            # Генерируем фразу без кэширования всех возможных комбинаций
            adjective = random.choice(config['adjectives'][category])
            name = random.choice(config['names'])
            phrase_template = random.choice(config['phrases'])
            phrase = phrase_template.format(adjective=adjective, name=name)
            
            param_phrases.append(phrase)
            annotations.append({
                "param": param,
                "label": category,
                "phrase": phrase,
                "code": random.choice(config["codes"])
            })
        
        if len(param_phrases) < min_params:
            continue
            
        # Формируем текст запроса
        params_text = param_phrases[0]
        for i in range(1, len(param_phrases)):
            connector = random.choice(CONNECTORS)
            params_text += connector + param_phrases[i]
        
        start_phrase = random.choice(START_PHRASES)
        purpose = random.choice(PURPOSES)
        text = start_phrase.format(params=params_text, purpose=purpose)
        
        params_str = "|".join([f"{ann['code']}:{ann['label']}" for ann in annotations])
        
        results.append({
            "text": text,
            "annotations": annotations,
            "params_str": params_str
        })
    
    return results

In [16]:
def generate_batch(batch_size, preloaded_data, min_params=2, max_params=10):
    """
    Генерирует батч примеров за один вызов (оптимизировано для скорости)
    """
    results = []
    param_cache = preloaded_data['param_cache']
    valid_params = preloaded_data['valid_params']
    connectors = preloaded_data['connectors']
    start_phrases = preloaded_data['start_phrases']
    purposes = preloaded_data['purposes']
    
    if not valid_params:
        return []
    
    for _ in range(batch_size):
        # Выбираем случайное количество параметров
        num_params = random.randint(min_params, min(max_params, len(valid_params)))
        selected_params = random.sample(valid_params, num_params)
        
        param_phrases = []
        annotations = []
        
        for param in selected_params:
            # Быстро выбираем случайную категорию для параметра
            categories = list(param_cache[param].keys())
            if not categories:
                continue
                
            category = random.choice(categories)
            # Быстро выбираем случайную фразу из кэша
            phrase_data = random.choice(param_cache[param][category])
            
            param_phrases.append(phrase_data["phrase"])
            annotations.append({
                "param": param,
                "label": category,
                "phrase": phrase_data["phrase"],
                "code": phrase_data["code"]
            })
        
        if len(param_phrases) < min_params:
            continue
            
        # Формируем текст запроса (векторизованно)
        params_text = param_phrases[0]
        for i in range(1, len(param_phrases)):
            connector = random.choice(connectors)
            params_text += connector + param_phrases[i]
        
        start_phrase = random.choice(start_phrases)
        purpose = random.choice(purposes)
        text = start_phrase.format(params=params_text, purpose=purpose)
        
        params_str = "|".join([f"{ann['code']}:{ann['label']}" for ann in annotations])
        
        results.append({
            "text": text,
            "annotations": annotations,
            "params_str": params_str
        })
    
    return results

In [17]:
def generate_ner_dataset_memory_efficient(df, param_config, n_samples=20_000, batch_size=500, n_workers=4):
    """
    Генерирует NER-датасет с оптимальным использованием памяти
    """
    # 1. Рассчитываем пороги на выборке
    print("Рассчитываем пороговые значения на выборке данных...")
    thresholds = calculate_thresholds_sampled(df, sample_size=50_000)
    
    # 2. Определяем валидные параметры
    valid_params = [p for p in param_config.keys() if p in df.columns]
    print(f"Валидные параметры для генерации: {len(valid_params)}")
    
    if not valid_params:
        print("⚠️ Нет валидных параметров для генерации!")
        return pd.DataFrame()
    
    # 3. Генерируем данные батчами
    print(f"🚀 Генерация {n_samples} примеров с использованием {n_workers} потоков...")
    
    total_batches = n_samples // batch_size + (1 if n_samples % batch_size else 0)
    all_results = []
    
    # Используем ThreadPoolExecutor вместо ProcessPoolExecutor для избежания копирования данных
    with ThreadPoolExecutor(max_workers=n_workers) as executor:
        # Создаем список задач для каждого батча
        futures = []
        for batch_idx in range(total_batches):
            current_batch_size = batch_size if batch_idx < total_batches - 1 else n_samples % batch_size
            if current_batch_size <= 0:
                continue
                
            future = executor.submit(
                generate_batch_memory_efficient,
                current_batch_size,
                df,
                param_config,
                thresholds
            )
            futures.append(future)
        
        # Последовательно получаем результаты с прогресс-баром
        for future in tqdm(futures, desc="Генерация данных"):
            try:
                batch_results = future.result()
                all_results.extend(batch_results)
                
                # Принудительно запускаем сборщик мусора после каждого батча
                gc.collect()
                
            except Exception as e:
                print(f"⚠️ Ошибка при генерации батча: {str(e)}")
                continue
    
    # 4. Обрезаем до точного количества запрошенных примеров
    all_results = all_results[:n_samples]
    
    print(f"✅ Сгенерировано {len(all_results)} примеров")
    return pd.DataFrame(all_results)

In [18]:
def cached_generate_dataset_memory_efficient(df, param_config, n_samples=20_000, cache_file="ner_dataset_cache.pkl"):
    """
    Генерирует датасет с кэшированием и оптимальным использованием памяти
    """
    # Проверяем наличие кэша
    if os.path.exists(cache_file):
        try:
            print(f"📥 Загружаем кэшированный датасет из {cache_file}")
            with open(cache_file, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"⚠️ Ошибка при загрузке кэша: {str(e)}. Генерируем новый датасет.")
    
    # Генерируем новый датасет с управлением памятью
    print(f"🔄 Генерируем новый датасет (оптимизировано для памяти)...")
    
    # Работаем только с нужными столбцами для экономии памяти
    needed_columns = [p for p in param_config.keys() if p in df.columns]
    if not needed_columns:
        needed_columns = df.columns[:10].tolist()  # Берем первые 10 столбцов как запасной вариант
        
    # Создаем маленький датафрейм только с нужными столбцами
    small_df = df[needed_columns].copy()
    
    # Очищаем память
    gc.collect()
    
    # Генерируем датасет
    dataset = generate_ner_dataset_memory_efficient(
        small_df, 
        param_config, 
        n_samples=n_samples,
        batch_size=200,  # Уменьшаем размер батча для экономии памяти
        n_workers=min(4, os.cpu_count() or 4)  # Не более 4 потоков
    )
    
    # Сохраняем в кэш только если датасет не пустой
    if not dataset.empty:
        try:
            with open(cache_file, 'wb') as f:
                pickle.dump(dataset, f)
            print(f"💾 Кэш сохранен в {cache_file}")
        except Exception as e:
            print(f"⚠️ Не удалось сохранить кэш: {str(e)}")
    
    return dataset

In [15]:
# Использование - оптимизированная версия для работы с ограниченной памятью
print("⚡ Генерируем оптимизированный датасет NER с управлением памятью...")
full_dataset = cached_generate_dataset_memory_efficient(
    df, 
    PARAM_CONFIG, 
    n_samples=100_000,  # Начинаем с меньшего количества
    cache_file="polymer_ner_dataset_100k.pkl"
)

print(f"✅ Сгенерировано {len(full_dataset)} примеров")
print(full_dataset.head())

⚡ Генерируем оптимизированный датасет NER с управлением памятью...


NameError: name 'cached_generate_dataset_memory_efficient' is not defined

In [68]:
full_dataset.to_csv('data/user_request_dataset/polymer_dataset_new_100к.csv', index=False, encoding='utf-8')

In [46]:
import ast
import json

def normalize_dataset(input_file, output_file):
    print(f"Загрузка {input_file}...")
    df = pd.read_csv(input_file)
    
    fixed_count = 0
    error_count = 0
    
    new_params_strs = []
    
    for index, row in df.iterrows():
        try:
            # 1. Парсим аннотации
            # Пробуем ast (для питоновских строк) или json
            try:
                annotations = ast.literal_eval(row['annotations'])
            except:
                try:
                    annotations = json.loads(row['annotations'])
                except:
                    # Если совсем битая строка, оставляем как есть или пустую
                    new_params_strs.append("")
                    error_count += 1
                    continue
            
            if not isinstance(annotations, list):
                new_params_strs.append("")
                continue

            # 2. Собираем новую строку params_str
            # Берем четкий 'param' из аннотации и 'label'
            # Формат: param1:label1|param2:label2
            parts = []
            for ann in annotations:
                if isinstance(ann, dict) and 'param' in ann and 'label' in ann:
                    clean_param = ann['param'] # Например "Tm" вместо "плавл"
                    clean_label = ann['label'] # "средняя"
                    parts.append(f"{clean_param}:{clean_label}")
            
            new_str = "|".join(parts)
            new_params_strs.append(new_str)
            fixed_count += 1
            
        except Exception as e:
            print(f"Ошибка в строке {index}: {e}")
            new_params_strs.append("")
            error_count += 1

    # 3. Заменяем столбец
    df['params_str'] = new_params_strs
    
    # 4. Сохраняем
    df.to_csv(output_file, index=False)
    print(f"Готово! Обработано строк: {fixed_count}. Ошибок: {error_count}")
    print(f"Файл сохранен как: {output_file}")
    
    # Показываем пример
    print("\nПример исправления:")
    print(df[['annotations', 'params_str']].head(1).values)

# Запуск исправления
normalize_dataset(
    'data/user_request_dataset/polymer_dataset_new_100к.csv', 
    'data/user_request_dataset/polymer_dataset_clean.csv'
)

Загрузка data/user_request_dataset/polymer_dataset_new_100к.csv...
Готово! Обработано строк: 99800. Ошибок: 0
Файл сохранен как: data/user_request_dataset/polymer_dataset_clean.csv

Пример исправления:
[["[{'param': 'Tm', 'label': 'средняя', 'phrase': 'с стандартная температура фазового перехода', 'code': 'плавл'}, {'param': 'epse_9.0', 'label': 'высокая', 'phrase': 'с выдающаяся диэлектрической проницаемостью на 9 ГГц', 'code': 'эпс_9'}]"
  'Tm:средняя|epse_9.0:высокая']]


In [47]:
full_dataset = pd.read_csv('data/user_request_dataset/polymer_dataset_clean.csv')

In [20]:
def filter_overlaps(entities):
    """
    Фильтрует пересекающиеся сущности, оставляя самые длинные.
    entities: список кортежей (start, end, label)
    """
    # Сортируем сущности по длине (от длинных к коротким)
    # Если длины равны, сортируем по начальной позиции
    sorted_entities = sorted(entities, key=lambda x: (x[1] - x[0], x[0]), reverse=True)
    
    filtered = []
    for candidate in sorted_entities:
        cand_start, cand_end, _ = candidate
        
        is_overlap = False
        for existing in filtered:
            ex_start, ex_end, _ = existing
            
            # Проверка пересечения отрезков:
            # max(start1, start2) < min(end1, end2) означает пересечение
            if max(cand_start, ex_start) < min(cand_end, ex_end):
                is_overlap = True
                break
        
        if not is_overlap:
            filtered.append(candidate)
            
    return filtered

def prepare_spacy_training_data(dataset_file):
    """Подготовка данных для обучения spaCy с фильтрацией пересечений"""
    # Загружаем датасет
    df = pd.read_csv(dataset_file)
    
    # Создаем обучающие данные в формате spaCy
    training_data = []
    
    for _, row in df.iterrows():
        text = row['text']
        raw_entities = [] # Сюда собираем всех кандидатов
        
        # --- Блок парсинга аннотаций (тот же, что и у вас) ---
        if 'annotations_str' in row and pd.notna(row['annotations_str']):
            try:
                annotations = ast.literal_eval(row['annotations_str'])
            except (ValueError, SyntaxError):
                try:
                    annotations = json.loads(row['annotations_str'])
                except (json.JSONDecodeError, TypeError):
                    print(f"Не удалось распарсить аннотации: {row['annotations_str']}")
                    continue
        elif 'annotations' in row and pd.notna(row['annotations']):
            try:
                annotations = ast.literal_eval(row['annotations'])
            except (ValueError, SyntaxError):
                try:
                    annotations = json.loads(row['annotations'])
                except (json.JSONDecodeError, TypeError):
                    print(f"Не удалось распарсить аннотации: {row['annotations']}")
                    continue
        else:
            # print("Не найдены аннотации в строке") # Можно закомментировать, чтобы не спамило
            continue
        
        if not isinstance(annotations, list):
            continue
        
        # Создаем кандидатов в аннотации
        for ann in annotations:
            if not isinstance(ann, dict):
                continue
                
            if 'phrase' not in ann or 'param' not in ann or 'label' not in ann:
                continue
            
            phrase = str(ann['phrase'])
            if not phrase: continue
            
            # !ВАЖНО: text.find находит только ПЕРВОЕ вхождение. 
            # Если в тексте фраза встречается дважды, а размечена вторая - это проблема.
            # Для простого решения пока оставим find, но имейте в виду этот риск.
            start = text.find(phrase)
            
            if start != -1:
                end = start + len(phrase)
                label = f"{ann['param']}_{ann['label'].upper()}"
                raw_entities.append((start, end, label))
        
        # --- ИСПРАВЛЕНИЕ: Фильтруем пересечения перед добавлением ---
        if raw_entities:
            clean_entities = filter_overlaps(raw_entities)
            training_data.append((text, {"entities": clean_entities}))
    
    # Создаем папку, если её нет
    os.makedirs('data/user_request_dataset', exist_ok=True)
    
    # Сохраняем в JSONL формате
    with open('data/user_request_dataset/spacy_training_data.jsonl', 'w', encoding='utf-8') as f:
        for text, entities_dict in training_data:
            json.dump({"text": text, "entities": entities_dict["entities"]}, f, ensure_ascii=False)
            f.write('\n')
    
    print(f"Подготовлено {len(training_data)} обучающих примеров")
    return training_data

In [14]:
# Обновленный класс датасета
class PropertyDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [15]:
class OptimizedPolymerNERTrainer:
    """Оптимизированный класс для обучения NER-модели"""
    
    def __init__(self, model_name="ru_core_news_sm"):
        """Инициализация модели"""
        # Проверяем доступность GPU
        self.use_gpu = torch.cuda.is_available()
        if self.use_gpu:
            print(f"Используется GPU: {torch.cuda.get_device_name(0)}")
        else:
            print("GPU не доступен, используется CPU")
        
        try:
            self.nlp = spacy.load(model_name)
        except OSError:
            print(f"Модель {model_name} не найдена. Создаем базовую модель.")
            self.nlp = spacy.blank("ru")
        
        # Добавляем NER компонент, если его нет
        if "ner" not in self.nlp.pipe_names:
            self.ner = self.nlp.add_pipe("ner")
        else:
            self.ner = self.nlp.get_pipe("ner")
    
    def add_labels_to_ner(self, training_data):
        """Добавление меток в NER компонент"""
        for _, annotations in training_data:
            for ent in annotations.get("entities", []):
                if len(ent) >= 3:  # Убедимся, что есть метка
                    self.ner.add_label(ent[2])  # ent[2] - это метка
    
    def train_model_fast(self, training_data, n_iter=20):
        """Быстрое обучение модели (5 эпох для тестирования)"""
        print(f"Начинаем быстрое обучение ({n_iter} эпох)...")
        
        # Добавляем метки
        self.add_labels_to_ner(training_data)
        
        # Убираем другие компоненты во время обучения
        pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
        unaffected_pipes = [pipe for pipe in self.nlp.pipe_names if pipe not in pipe_exceptions]
        
        # Быстрое обучение
        with self.nlp.disable_pipes(*unaffected_pipes):  # Только обучаем NER
            # Получаем оптимизатор
            optimizer = self.nlp.begin_training()
            
            # Увеличиваем размер батча для ускорения
            batch_size = 8  # Увеличен с 4 до 8
            
            for i in range(n_iter):
                print(f"Эпоха {i+1}/{n_iter}")
                losses = {}
                random.shuffle(training_data)
                
                # Используем большие батчи для ускорения
                batches = spacy.util.minibatch(
                    training_data, 
                    size=spacy.util.compounding(8.0, 64.0, 1.001)  # Увеличен минимальный размер
                )
                
                for batch in batches:
                    examples = []
                    for text, annotations in batch:
                        doc = self.nlp.make_doc(text)
                        example = Example.from_dict(doc, annotations)
                        examples.append(example)
                    
                    # Уменьшаем drop для более стабильного обучения
                    self.nlp.update(examples, drop=0.3, losses=losses, sgd=optimizer)
                
                print(f"Потери: {losses}")
                
                # Ранний выход, если потери очень маленькие
                if losses.get('ner', 0) < 0.01:
                    print("Достигнуты минимальные потери, останавливаем обучение")
                    break
        
        return self.nlp
    
    def save_model(self, model_path):
        """Сохранение обученной модели"""
        self.nlp.to_disk(model_path)
        print(f"Модель сохранена в {model_path}")

In [16]:
import torch
print("CUDA доступна:", torch.cuda.is_available())
print("torch.cuda.current_device():", torch.cuda.current_device())
print("torch.cuda.get_device_name(0):", torch.cuda.get_device_name(0))
print("torch.cuda.memory_allocated():", torch.cuda.memory_allocated())


CUDA доступна: True
torch.cuda.current_device(): 0
torch.cuda.get_device_name(0): NVIDIA GeForce RTX 3060 Laptop GPU
torch.cuda.memory_allocated(): 0


In [21]:
def train_with_progress_monitoring():
    """Обучение с мониторингом прогресса"""
    start_time = time.time()
    
    try:
        # Подготовка данных
        training_data = prepare_spacy_training_data(full_dataset)
        
        if not training_data:
            print("Нет обучающих данных!")
            return None
        
        print(f"Начинаем обучение на {len(training_data)} примерах...")
        print("Это займет несколько минут...")
        
        # Создание и обучение модели
        trainer = OptimizedPolymerNERTrainer()
        model = trainer.train_model_fast(training_data, n_iter=20)
        
        # Создаем директорию для модели
        import os
        os.makedirs("models", exist_ok=True)
        
        # Сохранение модели
        trainer.save_model("models/nlp_model_fast")
        
        end_time = time.time()
        training_time = end_time - start_time
        print(f"Обучение завершено за {training_time:.2f} секунд!")
        
        return model
        
    except Exception as e:
        print(f"Ошибка при обучении: {e}")
        import traceback
        traceback.print_exc()
        return None

# Быстрый запуск
model = train_with_progress_monitoring()

Подготовлено 99800 обучающих примеров
Начинаем обучение на 99800 примерах...
Это займет несколько минут...
Используется GPU: NVIDIA GeForce RTX 3060 Laptop GPU
Модель ru_core_news_sm не найдена. Создаем базовую модель.
Начинаем быстрое обучение (20 эпох)...
Эпоха 1/20
Потери: {'ner': np.float32(892923.06)}
Эпоха 2/20
Потери: {'ner': np.float32(618089.4)}
Эпоха 3/20
Потери: {'ner': np.float32(604502.06)}
Эпоха 4/20
Потери: {'ner': np.float32(410091.25)}
Эпоха 5/20
Потери: {'ner': np.float32(218867.47)}
Эпоха 6/20
Потери: {'ner': np.float32(182427.06)}
Эпоха 7/20
Потери: {'ner': np.float32(172708.69)}
Эпоха 8/20
Потери: {'ner': np.float32(175065.67)}
Эпоха 9/20
Потери: {'ner': np.float32(167125.36)}
Эпоха 10/20
Потери: {'ner': np.float32(164647.94)}
Эпоха 11/20
Потери: {'ner': np.float32(166184.69)}
Эпоха 12/20
Потери: {'ner': np.float32(161960.22)}
Эпоха 13/20
Потери: {'ner': np.float32(164984.02)}
Эпоха 14/20
Потери: {'ner': np.float32(163183.56)}
Эпоха 15/20
Потери: {'ner': np.float32

In [58]:
class PolymerParameterExtractor:
    """Класс для извлечения параметров из пользовательских запросов"""
    
    def __init__(self, model_path="models/nlp_model_fast"):
        """Загрузка обученной модели"""
        try:
            self.nlp = spacy.load(model_path)
        except OSError:
            print("Модель не найдена. Используется базовая модель.")
            self.nlp = spacy.load("ru_core_news_sm")
    
    def extract_parameters(self, text):
        """Извлечение параметров из текста"""
        doc = self.nlp(text)
        parameters = {}
        
        for ent in doc.ents:
            # Парсим метку формата PARAM_CATEGORY
            if '_' in ent.label_:
                param, category = ent.label_.split('_', 1)
                parameters[param] = {
                    'category': category.lower(),
                    'value': ent.text,
                    'start': ent.start_char,
                    'end': ent.end_char
                }
        
        return parameters
    
    def get_parameter_info(self):
        """Получение информации о параметрах"""
        return {
            "Egc": "Энергия группового вклада (кДж/моль)",
            "Egb": "Энергия вклада связи (кДж/моль)",
            "Eib": "Энергия внутренней связи (кДж/моль)",
            "CED": "Плотность когезионной энергии (Дж/см³)",
            "Ei": "Энергия ионизации (эВ)",
            "Eea": "Электронное сродство (эВ)",
            "Eat": "Энергия атомизации (кДж/моль)",
            "nc": "Количество атомов углерода",
            "ne": "Количество электронов",
            "Xc": "Степень кристалличности (%)",
            "Xe": "Плотность сшивки (моль/м³)",
            "epse_6.0": "Диэлектрическая проницаемость при 6.0 ГГц",
            "epsc": "Статическая диэлектрическая проницаемость (0 Гц)",
            "epse_3.0": "Диэлектрическая проницаемость при 3.0 ГГц",
            "epse_1.78": "Диэлектрическая проницаемость при 1.78 ГГц",
            "epse_15.0": "Диэлектрическая проницаемость при 15.0 ГГц",
            "epse_4.0": "Диэлектрическая проницаемость при 4.0 ГГц",
            "epse_5.0": "Диэлектрическая проницаемость при 5.0 ГГц",
            "epse_2.0": "Диэлектрическая проницаемость при 2.0 ГГц",
            "epse_9.0": "Диэлектрическая проницаемость при 9.0 ГГц",
            "epse_7.0": "Диэлектрическая проницаемость при 7.0 ГГц",
            "epsb": "Диэлектрическая прочность пробоя (кВ/мм)",
            "TSb": "Прочность при разрыве (МПа)",
            "TSy": "Прочность при текучести (МПа)",
            "YM": "Модуль Юнга (МПа)",
            "permCH4": "Проницаемость для CH₄ (Бэрр)",
            "permCO2": "Проницаемость для CO₂ (Бэрр)",
            "permH2": "Проницаемость для H₂ (Бэрр)",
            "permO2": "Проницаемость для O₂ (Бэрр)",
            "permN2": "Проницаемость для N₂ (Бэрр)",
            "permHe": "Проницаемость для He (Бэрр)",
            "Cp": "Удельная теплоёмкость (Дж/(г·К))",
            "Td": "Температура термического разложения (К)",
            "Tg": "Температура стеклования (К)",
            "Tm": "Температура плавления (К)",
            "rho": "Плотность (г/см³)",
            "LOI": "Порог кислородного индекса (%)"
        }

# Использование
extractor = PolymerParameterExtractor()

In [60]:
class ParameterRangeConverter:
    """Класс для преобразования категорий в числовые диапазоны"""
    
    def __init__(self, training_datasets):
        """Инициализация с исходными датасетами для определения диапазонов"""
        self.thresholds = self._calculate_thresholds(training_datasets)
    
    def _calculate_thresholds(self, datasets):
        """Вычисление порогов для каждого параметра"""
        thresholds = {}
        
        # Объединяем все датасеты для вычисления глобальных порогов
        all_data = pd.concat(datasets, ignore_index=True)
        
        # Вычисляем пороги для всех параметров
        for param in set(common_columns):
            if param in all_data.columns:
                valid_values = all_data[param].dropna()
                if len(valid_values) > 0:
                    low = valid_values.quantile(0.33)
                    high = valid_values.quantile(0.67)
                    thresholds[param] = {
                        'low': low,
                        'high': high,
                        'min': valid_values.min(),
                        'max': valid_values.max(),
                        'mean': valid_values.mean(),
                        'std': valid_values.std()
                    }
        
        return thresholds
    
    def get_range_for_category(self, parameter, category):
        """Получение числового диапазона для параметра и категории"""
        if parameter not in self.thresholds:
            return None
        
        param_info = self.thresholds[parameter]
        
        if category == 'низкая':
            return {
                'min': param_info['min'],
                'max': param_info['low'],
                'description': f"Низкие значения {parameter}"
            }
        elif category == 'средняя':
            return {
                'min': param_info['low'],
                'max': param_info['high'],
                'description': f"Средние значения {parameter}"
            }
        elif category == 'высокая':
            return {
                'min': param_info['high'],
                'max': param_info['max'],
                'description': f"Высокие значения {parameter}"
            }
        else:
            return None
    
    def get_detailed_recommendation(self, parameter, category):
        """Получение детальной рекомендации"""
        if parameter not in self.thresholds:
            return "Параметр не найден"
        
        param_info = self.thresholds[parameter]
        range_info = self.get_range_for_category(parameter, category)
        
        if not range_info:
            return "Неверная категория"
        
        # Форматируем значения в зависимости от порядка величины
        if abs(param_info['max'] - param_info['min']) > 100:
            min_val = f"{range_info['min']:.2f}"
            max_val = f"{range_info['max']:.2f}"
        elif abs(param_info['max'] - param_info['min']) > 1:
            min_val = f"{range_info['min']:.3f}"
            max_val = f"{range_info['max']:.3f}"
        else:
            min_val = f"{range_info['min']:.4f}"
            max_val = f"{range_info['max']:.4f}"
        
        return {
            'parameter': parameter,
            'category': category,
            'range': f"[{min_val}, {max_val}]",
            'min_value': float(range_info['min']),
            'max_value': float(range_info['max']),
            'description': range_info['description']
        }

# Инициализация конвертера
converter = ParameterRangeConverter([df])

In [61]:
class PolymerRecommendationSystem:
    """Основной класс для обработки пользовательских запросов"""
    
    def __init__(self, ner_model_path="models/nlp_model_fast", training_datasets=None):
        """Инициализация системы"""
        self.extractor = PolymerParameterExtractor(ner_model_path)
        self.converter = ParameterRangeConverter(training_datasets) if training_datasets else None
    
    def process_request(self, user_request):
        """Обработка пользовательского запроса"""
        print(f"Обработка запроса: {user_request}")
        
        # 1. Извлекаем параметры
        parameters = self.extractor.extract_parameters(user_request)
        
        if not parameters:
            return {
                "error": "Не удалось извлечь параметры из запроса",
                "extracted_text": user_request
            }
        
        # 2. Преобразуем в числовые диапазоны
        recommendations = {}
        for param, info in parameters.items():
            category = info['category']
            if self.converter:
                recommendation = self.converter.get_detailed_recommendation(param, category)
                recommendations[param] = {
                    'category': category,
                    'extracted_phrase': info['value'],
                    'recommendation': recommendation
                }
            else:
                recommendations[param] = {
                    'category': category,
                    'extracted_phrase': info['value'],
                    'recommendation': f"Для параметра {param} с категорией {category}"
                }
        
        return {
            "original_request": user_request,
            "extracted_parameters": parameters,
            "recommendations": recommendations,
            "success": True
        }
    
    def format_output(self, result):
        """Форматирование результата для вывода"""
        if not result.get('success', False):
            return f"Ошибка: {result.get('error', 'Неизвестная ошибка')}"
        
        output = []
        output.append("=== РЕКОМЕНДАЦИИ ПО ПОЛИМЕРУ ===")
        output.append(f"Запрос: {result['original_request']}")
        output.append("")
        output.append("Извлеченные параметры:")
        
        for param, info in result['recommendations'].items():
            output.append(f"  {param}:")
            output.append(f"    Категория: {info['category']}")
            output.append(f"    Извлечено: {info['extracted_phrase']}")
            if isinstance(info['recommendation'], dict):
                rec = info['recommendation']
                output.append(f"    Диапазон значений: {rec['range']}")
                output.append(f"    Описание: {rec['description']}")
            else:
                output.append(f"    Рекомендация: {info['recommendation']}")
            output.append("")
        
        return "\n".join(output)

In [26]:
def save_model_artifacts(model, converter, extractor, save_dir="models/nlp_model_fast/ner"):
    """
    Сохранение всех компонентов системы (NLP модель + вспомогательные классы)
    
    Args:
        model: Экземпляр OptimizedPolymerNERTrainer
        converter: Объект конвертера (например, для перевода значений)
        extractor: Объект экстрактора (логика извлечения)
        save_dir: Путь к папке для сохранения
    """
    # Создаем директорию, если её нет
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    print(f"Сохранение артефактов в: {save_dir}...")

    try:
        # 1. Сохраняем модель spaCy
        nlp = model.nlp if hasattr(model, 'nlp') else model
        nlp_path = os.path.join(save_dir, "spacy_ner_model")
        nlp.to_disk(nlp_path)
        print(f"✓ NLP модель сохранена в {nlp_path}")

        # 2. Сохраняем Python объекты (Converter и Extractor) через pickle
        # Мы сохраняем их в один файл словарем, чтобы проще было грузить
        artifacts = {
            "converter": converter,
            "extractor": extractor,
            # Можно добавить метаданные версии
            "version": "1.0",
            "model_type": "polymer_ner"
        }
        
        artifacts_path = os.path.join(save_dir, "artifacts.pkl")
        with open(artifacts_path, "wb") as f:
            pickle.dump(artifacts, f)
            
        print(f"✓ Вспомогательные компоненты (converter, extractor) сохранены в {artifacts_path}")
        print("Все компоненты системы успешно сохранены.")
        
    except Exception as e:
        print(f"CRITICAL ERROR при сохранении модели: {e}")
        raise e

In [66]:
# Создаем систему
system = PolymerRecommendationSystem(
    ner_model_path="models/nlp_model_fast",
    training_datasets=[df]
)

# Обрабатываем запрос
request = "Интересует полимер с с маленьким индексом горючести, что важно для с стандартной статической диэлектрической проницаемости, еще , что необходимо низкая проницаемость для гелия, что важно с выдающимся энергетическим вкладом связей, и низкой плотностью, чтобы выдерживать высокие нагрузки"
result = system.process_request(request)
print(system.format_output(result))

Обработка запроса: Интересует полимер с с маленьким индексом горючести, что важно для с стандартной статической диэлектрической проницаемости, еще , что необходимо низкая проницаемость для гелия, что важно с выдающимся энергетическим вкладом связей, и низкой плотностью, чтобы выдерживать высокие нагрузки
=== РЕКОМЕНДАЦИИ ПО ПОЛИМЕРУ ===
Запрос: Интересует полимер с с маленьким индексом горючести, что важно для с стандартной статической диэлектрической проницаемости, еще , что необходимо низкая проницаемость для гелия, что важно с выдающимся энергетическим вкладом связей, и низкой плотностью, чтобы выдерживать высокие нагрузки

Извлеченные параметры:
  LOI:
    Категория: низкая
    Извлечено: с маленьким индексом горючести
    Диапазон значений: [15.560, 29.495]
    Описание: Низкие значения LOI

  epsc:
    Категория: низкая
    Извлечено: с стандартной статической диэлектрической проницаемости
    Диапазон значений: [2.926, 4.237]
    Описание: Низкие значения epsc

  permHe:
    Кат